# Tutorial for DeepAugment

- Github: https://github.com/barisozmen/deepaugment
- Slides: https://bit.ly/deepaugmentslides
- PyPI: https://pypi.org/project/deepaugment/


In this tutorial, we will:
1. Install
2. Import
3. Setup and configure
4. Optimizer augmentations
5. Print found augmentation policies
6. Create image-generator using augmentation policies
7. Use the image-generator on a full-model

**Important Note:** This notebook is only for showing usage of the library. Configurations are made as running the notebook will take 10-20 minutes (using TPU). Therefore this is not a proper analysis with useful results. A proper analysis would take 5-10 hours. For a proper analysis, change configurations as following:
* **child_epochs:** +100 (Section 3)
* **optimizer iterations:** +100 (Section 4)
* **model:** wrn_28_10 or InceptionV3 (Section 7)

See [slides](http://bit.ly/deepaugmentslides) for results with CIFAR-10 and iNaturalist images, where DeepAugment was ran for +4 hours on AWS p3.x2large instance

### Contact
Baris Ozmen, hbaristr@gmail.com

## 1. Install

If running in a cloud notebook you can also try to install the dependencies with pip and clone the rest of the package. For local installs I recommend to skip this and refer to the readme

In [ ]:
#!pip install --upgrade pip

In [ ]:
#!pip install -r requirements.txt

### Installed version

## 2.Import

In [1]:
from deepaugment.deepaugment import DeepAugment

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-06-23 16:58:48.190980: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-23 16:58:48.191107: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-06-23 16:58:48.525440: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-23 16:58:48.525464: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## 3. A simple setup of DeepAugment

Number of child epochs are set as 10 in order to make a quick run. For a thorough analysis, set `child_epochs` >=50

In [7]:
type(y_train)

numpy.ndarray

In [2]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# child_epochs set to 10 for a quick run, but it should be >=50 for a proper analysis
my_config = {
    "child_epochs":2,
    "opt_samples":1
}

# X_train.shape -> (N, M, M, 3)
# y_train.shape -> (N)
deepaug = DeepAugment(images=x_train, labels=y_train, config=my_config)

Using 2000 training images


2022-06-23 17:10:30.274829: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-23 17:10:30.275721: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


BasicCNN model built as child model.
 Model summary:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
   

2022-06-23 17:10:31.852685: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-06-23 17:10:32.681644: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-06-23 17:10:40.697920: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


188/188 - 9s - loss: 2.0722 - accuracy: 0.2273 - val_loss: 1.8338 - val_accuracy: 0.3170 - 9s/epoch - 48ms/step
Epoch 2/2
188/188 - 8s - loss: 1.7127 - accuracy: 0.3760 - val_loss: 2.1802 - val_accuracy: 0.2980 - 8s/epoch - 42ms/step
fit()'s runtime:  17.5918 sec.
0, 0.6924999952316284, ['<identity>', 0, '<identity>', 0, '<identity>', 0, '<identity>', 0, '<identity>', 0, '<identity>', 0, '<identity>', 0, '<identity>', 0, '<identity>', 0, '<identity>', 0]


## 4. Optimize for 1 iteration (normally 100-300 iterations needed)
- Each iteration takes ~30 secs on AWS p3.x2large (V100 GPU), therefore a proper optimization with 300 iterations would take ~2.5 hours and cost ~8$.

    - AWS instance p3.x2large cost: 3.09$/h  

In [3]:
# number of iterations set to 2 (set it >=100 for a proper analysis)
best_policies = deepaug.optimize(10)
#x_portion_aug_max is 305

trial: 1 
 ['<Brightness>', 0, '<Brightness>', 3, '<ShearY>', 19, '<Sharpness>', 4, '<Solarize>', 24, '<TranslateY>', 26, '<Posterize>', 7, '<Contrast>', 24, '<Color>', 25, '<identity>', 9]
before augmenting sace train dataset
Länge Hyperparams:20
load_pre_augment_weights()'s runtime:  0.0171 sec.
Epoch 1/2
188/188 - 6s - loss: 2.1695 - accuracy: 0.1892 - val_loss: 6.2111 - val_accuracy: 0.1650 - 6s/epoch - 33ms/step
Epoch 2/2
188/188 - 6s - loss: 1.9802 - accuracy: 0.2797 - val_loss: 7.6470 - val_accuracy: 0.1890 - 6s/epoch - 32ms/step
fit()'s runtime:  12.4463 sec.
1, 0.8229999914765358, ['<Brightness>', 0, '<Brightness>', 3, '<ShearY>', 19, '<Sharpness>', 4, '<Solarize>', 24, '<TranslateY>', 26, '<Posterize>', 7, '<Contrast>', 24, '<Color>', 25, '<identity>', 9]
trial: 2 
 ['<Sharpness>', 16, '<ShearY>', 30, '<AutoContrast>', 0, '<TranslateY>', 24, '<ShearY>', 29, '<ShearX>', 14, '<ShearX>', 1, '<Equalize>', 0, '<Contrast>', 20, '<Posterize>', 30]
before augmenting sace train datase

## 5. See found best policies

In [4]:
best_policies

,trial_no,A_aug1_type,A_aug1_magnitude,A_aug2_type,A_aug2_magnitude,B_aug1_type,B_aug1_magnitude,B_aug2_type,B_aug2_magnitude,C_aug1_type,...,D_aug1_type,D_aug1_magnitude,D_aug2_type,D_aug2_magnitude,E_aug1_type,E_aug1_magnitude,E_aug2_type,E_aug2_magnitude,mean_late_val_acc,expected_accuracy_increase(%)
0,0,<identity>,0,<identity>,0,<identity>,0,<identity>,0,<identity>,...,<identity>,0,<identity>,0,<identity>,0,<identity>,0,0.308,0.0
4,2,<Sharpness>,16,<ShearY>,30,<AutoContrast>,0,<TranslateY>,24,<ShearY>,...,<ShearX>,1,<Equalize>,0,<Contrast>,20,<Posterize>,30,0.254,-5.4
6,3,<ShearX>,23,<Rotate>,0,<identity>,21,<Equalize>,8,<Brightness>,...,<Color>,24,<TranslateX>,7,<identity>,3,<Sharpness>,11,0.226,-8.2
17,8,<TranslateX>,24,<Rotate>,12,<Sharpness>,0,<ShearY>,13,<AutoContrast>,...,<Sharpness>,16,<Brightness>,21,<TranslateY>,16,<ShearX>,22,0.200,-10.8
10,5,<AutoContrast>,21,<Solarize>,5,<Equalize>,0,<Sharpness>,30,<AutoContrast>,...,<TranslateY>,2,<Contrast>,16,<Brightness>,9,<identity>,10,0.188,-12.0
18,9,<Contrast>,19,<Sharpness>,0,<Brightness>,27,<identity>,21,<ShearX>,...,<ShearX>,8,<AutoContrast>,11,<AutoContrast>,29,<Brightness>,22,0.178,-13.0
3,1,<Brightness>,0,<Brightness>,3,<ShearY>,19,<Sharpness>,4,<Solarize>,...,<Posterize>,7,<Contrast>,24,<Color>,25,<identity>,9,0.177,-13.1
20,10,<Posterize>,15,<Color>,2,<ShearX>,3,<Contrast>,10,<Solarize>,...,<ShearX>,2,<identity>,0,<Color>,8,<identity>,0,0.176,-13.2
14,7,<Sharpness>,19,<Sharpness>,22,<Solarize>,19,<Color>,19,<identity>,...,<Sharpness>,3,<Sharpness>,11,<TranslateY>,8,<ShearX>,8,0.169,-13.9
12,6,<Rotate>,26,<Color>,27,<AutoContrast>,2,<TranslateX>,30,<identity>,...,<Rotate>,27,<TranslateX>,26,<Brightness>,10,<Rotate>,8,0.149,-15.9


## 6. Create Image Generator by found best policies

In [8]:
import keras
image_gen = deepaug.image_generator_with_top_policies(x_train, keras.utils.np_utils.to_categorical(y_train))

## 7. Use DeepAugmented images on your model

Let's try  BasicCNN (Child CNN) model which can be created by modules in the package (assuming that your full model is same with BasicCNN). For a proper analysis use WideResNet-28-10, or better, your own model.

In [9]:
from deepaugment.childcnn import ChildCNN
import logging

cnn_config = {"model":"basicCNN", "logging":logging}
full_model = ChildCNN(input_shape=x_train.shape[1:], num_classes=10, config=cnn_config)

BasicCNN model built as child model.
 Model summary:
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 activation_6 (Activation)   (None, 32, 32, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 30, 30, 32)        9248      
                                                                 
 activation_7 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 15, 15, 32)        0         
 

In [10]:
BATCH_SIZE = 64

full_model.model.fit_generator(
    image_gen,
    validation_data=(x_test, keras.utils.np_utils.to_categorical(y_test)),
    steps_per_epoch=len(x_train) // BATCH_SIZE,
    epochs=5,
    shuffle=True,
    verbose=2
)

Policies are:
[{'A_aug1_type': 'rotate', 'A_aug1_magnitude': 0.0, 'A_aug2_type': 'rotate', 'A_aug2_magnitude': 0.0, 'B_aug1_type': 'rotate', 'B_aug1_magnitude': 0.0, 'B_aug2_type': 'rotate', 'B_aug2_magnitude': 0.0, 'C_aug1_type': 'rotate', 'C_aug1_magnitude': 0.0, 'C_aug2_type': 'rotate', 'C_aug2_magnitude': 0.0, 'D_aug1_type': 'rotate', 'D_aug1_magnitude': 0.0, 'D_aug2_type': 'rotate', 'D_aug2_magnitude': 0.0, 'E_aug1_type': 'rotate', 'E_aug1_magnitude': 0.0, 'E_aug2_type': 'rotate', 'E_aug2_magnitude': 0.0}, {'A_aug1_type': 'dropout', 'A_aug1_magnitude': 0.801, 'A_aug2_type': 'coarse-dropout', 'A_aug2_magnitude': 0.679, 'B_aug1_type': 'fog', 'B_aug1_magnitude': 0.582, 'B_aug2_type': 'coarse-dropout', 'B_aug2_magnitude': 0.759, 'C_aug1_type': 'rotate', 'C_aug1_magnitude': 0.474, 'C_aug2_type': 'shear', 'C_aug2_magnitude': 0.737, 'D_aug1_type': 'translate-x', 'D_aug1_magnitude': 0.135, 'D_aug2_type': 'sharpen', 'D_aug2_magnitude': 0.15, 'E_aug1_type': 'translate-x', 'E_aug1_magnitude'

/var/folders/fd/j59lqzvj66n4r0_dyt15l1bw0000gn/T/ipykernel_65786/4149183280.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  full_model.model.fit_generator(
2022-04-02 22:59:15.802755: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-04-02 23:00:43.033169: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


781/781 - 91s - loss: 2.3305 - accuracy: 0.1128 - val_loss: 2.1359 - val_accuracy: 0.2171 - 91s/epoch - 116ms/step
Epoch 2/5
781/781 - 68s - loss: 2.1339 - accuracy: 0.2048 - val_loss: 1.9447 - val_accuracy: 0.3076 - 68s/epoch - 88ms/step
Epoch 3/5
781/781 - 43s - loss: 2.0262 - accuracy: 0.2508 - val_loss: 1.8860 - val_accuracy: 0.3272 - 43s/epoch - 55ms/step
Epoch 4/5
781/781 - 42s - loss: 1.9685 - accuracy: 0.2703 - val_loss: 1.9551 - val_accuracy: 0.2821 - 42s/epoch - 54ms/step
Epoch 5/5
781/781 - 42s - loss: 1.9191 - accuracy: 0.2950 - val_loss: 1.7552 - val_accuracy: 0.3725 - 42s/epoch - 54ms/step
